In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import scipy.optimize as sc
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def change(close, mean):
    last = close[-1]
    chg = mean/last - 1
    return chg

### Optimization

In [3]:
def get_data(stock_list, start, end):
    stock_list.sort()
    data = pdr.get_data_yahoo(stock_list, start, end)
    return data['Close']
def stock_performance(close):
    returns = close.pct_change()
    mean_returns = returns.mean()
    cov = returns.cov()
    return mean_returns, cov

def portfolio_performance(W, mean_returns, cov):
    W = np.asarray(W)
    portfolio_returns = (np.dot(W, mean_returns) * 252)
    portfolio_risk = np.sqrt(W.T @ cov @ W) * np.sqrt(252)
    return portfolio_returns, portfolio_risk  

############################
def negative_sharpe_ratio(W, mean_returns, cov, risk_free_rate):
    portfolio_return, portfolio_risk = portfolio_performance(W, mean_returns, cov)
    neg_sharpe_ratio = -(portfolio_return - risk_free_rate)/portfolio_risk
    return neg_sharpe_ratio

def optimize_portfolio(mean_returns, cov, upper_bound, risk_free_rate):
    """
    returns
    -------
    sharpe_ratio, optimal_weights
    """
    #assign random weights
    np.random.seed(1)
    W = np.random.random(len(mean_returns))
    W = [weight/ np.sum(W) for weight in W]
    
    #add bounds
    bound = (0,upper_bound)
    bounds = tuple(bound for w in range(len(W)))
    
    #constraint 
    def constraint(W):
        return np.sum(W) - 1
    constraint_set = [{'type': 'eq', 'fun': constraint}]
    
    #minimize negative SharpeRatio
    result = sc.minimize(negative_sharpe_ratio,
                        W,
                        args=(mean_returns, cov, risk_free_rate),
                        method='SLSQP',
                        bounds= bounds,
                        constraints=constraint_set)
    neg_sharpe_ratio, optimal_weights = result['fun'], result['x'].round(4)             
    return -neg_sharpe_ratio, optimal_weights   


######################
def minimum_risk_portfolio(mean_returns, cov, upper_bound, risk_free_rate):
    """
    returns
    -------
    sharpe_ratio, optimal_weights"""
     #assign random weights
    np.random.seed(1)
    W = np.random.random(len(mean_returns))
    W = [weight/ np.sum(W) for weight in W]

    #add bounds
    bound = (0,upper_bound)
    bounds = tuple(bound for w in range(len(W)))

    #constraint 
    def constraint(W):
        return np.sum(W) - 1
    constraint_set = [{'type': 'eq', 'fun': constraint}]
    
    def portfolio_variance(W,cov):
        return (np.sqrt(W.T @ cov @ W) * np.sqrt(252))

    result = sc.minimize(portfolio_variance,
                        W,
                        args = (cov),
                        bounds = bounds,
                        constraints = constraint_set,
                        method = 'SLSQP')

    sharpe_ratio, optimal_weights = result['fun'], result['x'].round(4)
    return -sharpe_ratio, optimal_weights


### Efficient market frontier

In [4]:
def simulate_portfolios(stock_list, start, end, simulations = 1000):
    
    close = get_data(stock_list, start, end)
    mean_returns, cov = stock_performance(close)
    
    returns = []
    risk = []
    weights = []
    for p in range(simulations):
        W = np.random.random(len(stock_list))
        W = W / np.sum(W)
        weights.append(W)
        
        portfolio_returns, portfolio_risk = portfolio_performance(W, mean_returns, cov)
        returns.append(portfolio_returns)
        risk.append(portfolio_risk)

    portfolios_weights = pd.DataFrame(weights, columns= stock_list)
    
    portfolios = pd.concat([pd.DataFrame({'Returns': returns, 'Risk': risk,}), portfolios_weights], axis = 1)
    return portfolios

def plot_EF(data,
            mean_returns,
            cov,
            stock_list,
            upper_bound,
            risk_free_rate,
            start,
            end,
            simulations = 10000):
            
    portfolios = simulate_portfolios(stock_list, start, end, simulations)
    
    #portfolios    
    fig = px.scatter(portfolios, portfolios['Risk'], portfolios['Returns'], hover_data=stock_list)
    
    #minimum varaince portfolio
    _, optimal_weights = minimum_risk_portfolio(mean_returns, cov, upper_bound, risk_free_rate)
    minimum_risk_portfolio_return, minimum_risk_portfolio_risk = portfolio_performance(optimal_weights, mean_returns, cov)
    #plot
    fig.add_trace(go.Scatter(x = [minimum_risk_portfolio_risk], y = [minimum_risk_portfolio_return],
                            mode = 'markers',
                            marker_symbol = 'star',
                            marker_size = 10,
                            marker = {'color' : 'red'}))

    #maximum sharpe ratio portfolio
    _, optimal_weights = optimize_portfolio(mean_returns, cov,upper_bound, risk_free_rate)
    optimum_portfolio_returns, optimum_portfolio_risk = portfolio_performance(optimal_weights, mean_returns, cov)
    #plot
    fig.add_trace(go.Scatter(x=[optimum_portfolio_risk], y=[optimum_portfolio_returns], mode = 'markers',
                         marker_symbol = 'star',
                         marker_size = 10,
                         marker = {'color' : 'red'}))
    fig.show() 

### Random walk simulation

In [5]:
def MC(close, steps, simulations, random_seed):
    daily_returns = np.log(1 + close.pct_change())
    mu = np.mean(daily_returns)
    var = np.var(daily_returns)
    sigma = np.std(daily_returns)
    drift = mu - 0.5*var

    returns = np.exp(drift + sigma * norm.ppf(np.random.rand(steps, simulations)))
    np.random.seed(random_seed)
    sims = np.zeros((steps, simulations))
    sims[0] = close.iloc[-1]

    for step in range(1,steps):
        sims[step] = sims[step-1] * returns[step]
    return sims


def monteCarlo(close, steps, simulations, random_seed):

    sims = MC(close=close, steps=steps, simulations=simulations, random_seed=random_seed)
    mcmean = np.mean(sims[-1])
    mcvar = np.var(sims[-1])
    return mcmean, mcvar 

In [6]:
def main(start, end, stock_list, upper_bound, simulations,  monte_carlo = False, steps = 90, random_seed = 1):
    """
    Paramters:
    ----------
    upper_bound: specifies upper bound for portfolio weights
    monte_carlo: whether to use monte carlo simulation to model stock price
    steps: number of days simualted in when using monte carlo
    """
    
    

    #bug fix
    yf.pdr_override()
    
    #get stock qoutes
    close = get_data(stock_list, start, end)

    mean_returns, cov = stock_performance(close)
    
    if monte_carlo: 

        mean_returns = []
        for i in range(close.shape[1]):
            c = close.iloc[:,i]
            mcmean, mcvar = monteCarlo(close=c, steps=steps,simulations=simulations,random_seed=random_seed)
            mean_returns.append(change(close=c, mean=mcmean))
        mean_returns = np.array(mean_returns)
    risk_free_rate = 0.04    
    #minimum variance porfolio    
    SR, optimal_weights = minimum_risk_portfolio(mean_returns, cov, upper_bound, risk_free_rate)
    portfolio_returns, portfolio_risk = portfolio_performance(optimal_weights, mean_returns, cov)

    print(f'Expected return: {portfolio_returns.round(3)}, Risk: {portfolio_risk.round(3)} with Sharpe Ratio:{SR.round(3)}\noptimal weights:')
    print(f'{close.columns.to_list()}\n{optimal_weights}')
    print('\n--------------\n')

    #maximum Sharpe Ratio portfolio
    SR, optimal_weights = optimize_portfolio(mean_returns, cov, upper_bound, risk_free_rate)
    portfolio_returns, portfolio_risk = portfolio_performance(optimal_weights, mean_returns, cov)

    print(f'Expected return: {portfolio_returns.round(3)}, Risk: {portfolio_risk.round(3)} with Sharpe Ratio:{SR.round(3)}\noptimal weights:')
    print(f'{close.columns.to_list()}\n{optimal_weights}')
    
    df = pd.DataFrame({"ticker":close.columns.to_list(), "weight": optimal_weights})
    best_weights = df.loc[df['weight']>0,:].reset_index(drop=True)
    
    # plot_EF(close,
    #         mean_returns,
    #         cov,
    #         close.columns.to_list(),
    #         upper_bound,
    #         risk_free_rate,
    #         start,
    #         end,
    #         simulations = 10000)
    
    return best_weights

In [7]:
long = pd.read_excel("C:\\Users\\k_abo\\Downloads\\ESG-F-Filter.xlsx", sheet_name = "Long")

stock_list1 = long['ticker'].to_list()

#time period
end = dt.datetime.now()
start = end - dt.timedelta(720)
simulations = 10000 
upper_bound = 0.2
steps = 90 
random_seed = 1
portfolio = main(start=start, end=end, stock_list=stock_list1, simulations=simulations, monte_carlo=False, upper_bound=upper_bound, steps=steps, random_seed=random_seed)
portfolio

[*********************100%***********************]  33 of 33 completed
Expected return: 0.1, Risk: 0.145 with Sharpe Ratio:-0.145
optimal weights:
['A', 'AMGN', 'AMP', 'APD', 'BLK', 'BSX', 'CAH', 'CDNS', 'CMCSA', 'COP', 'CSCO', 'CVS', 'EBAY', 'ECL', 'FCX', 'HES', 'LYB', 'M', 'MOS', 'MPC', 'MRK', 'MS', 'MSFT', 'MTD', 'OMC', 'PKG', 'PLD', 'PVH', 'STT', 'TGT', 'TSCO', 'VTR', 'WELL']
[0.     0.2    0.     0.0576 0.     0.0494 0.0455 0.     0.0409 0.0231
 0.0296 0.0839 0.     0.     0.     0.     0.     0.     0.     0.008
 0.2    0.     0.     0.     0.0436 0.0759 0.     0.     0.     0.
 0.0418 0.     0.1008]

--------------

Expected return: 0.317, Risk: 0.202 with Sharpe Ratio:1.373
optimal weights:
['A', 'AMGN', 'AMP', 'APD', 'BLK', 'BSX', 'CAH', 'CDNS', 'CMCSA', 'COP', 'CSCO', 'CVS', 'EBAY', 'ECL', 'FCX', 'HES', 'LYB', 'M', 'MOS', 'MPC', 'MRK', 'MS', 'MSFT', 'MTD', 'OMC', 'PKG', 'PLD', 'PVH', 'STT', 'TGT', 'TSCO', 'VTR', 'WELL']
[0.     0.     0.     0.     0.     0.     0.1135 0.1399

,ticker,weight
0,CAH,0.1135
1,CDNS,0.1399
2,COP,0.0670
3,HES,0.1021
4,MOS,0.0266
5,MPC,0.2000
6,MRK,0.2000
7,MTD,0.0542
8,TSCO,0.0967


In [13]:
uncertain = pd.read_excel("C:\\Users\\k_abo\\Downloads\\ESG-F-Filter.xlsx", sheet_name = "uncertain")
stock_list2 = uncertain['ticker2'].to_list()


#time period
end = dt.datetime.now()
start = end - dt.timedelta(720)
simulations = 10000 
upper_bound = 0.2
steps = 90 
random_seed = 1
portfolio = main(start=start, end=end, stock_list=stock_list2, simulations=simulations, monte_carlo=False, upper_bound=upper_bound, steps=steps, random_seed=random_seed)
portfolio

[*********************100%***********************]  59 of 59 completed
Expected return: 5.293, Risk: 0.13 with Sharpe Ratio:-0.13
optimal weights:
['ABC', 'ABT', 'ALB', 'AMCR', 'ARE', 'AVB', 'AXP', 'BALL', 'BBY', 'BIIB', 'BMY', 'BXP', 'C', 'CF', 'CNP', 'COF', 'CVS', 'CVX', 'DHR', 'DVN', 'ED', 'EQIX', 'ES', 'FMC', 'HAS', 'HST', 'IRM', 'JNJ', 'JPM', 'JWN', 'KEYS', 'KIM', 'KSS', 'LOW', 'M', 'MAC', 'MRO', 'NI', 'NWL', 'OXY', 'PEG', 'PFE', 'PPG', 'REG', 'RSG', 'SBAC', 'SHW', 'SRE', 'SYF', 'TGT', 'UA', 'UAA', 'V', 'VFC', 'WBA', 'WEC', 'WFC', 'WM', 'XEL']
[0.0386 0.     0.     0.     0.     0.     0.     0.0098 0.     0.
 0.2    0.     0.     0.0287 0.     0.     0.0011 0.0695 0.     0.
 0.1638 0.     0.     0.     0.0521 0.0097 0.     0.2    0.0097 0.
 0.0067 0.     0.     0.0305 0.     0.     0.     0.     0.     0.
 0.     0.0042 0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.0185 0.     0.     0.     0.     0.1571 0.    ]

--------------

Expected return: 21.069, Ris

,ticker,weight
0,ABC,0.2000
1,ALB,0.0208
2,CF,0.0548
3,DVN,0.1283
4,ED,0.2000
5,IRM,0.1023
6,MRO,0.0658
7,OXY,0.0280
8,RSG,0.2000


In [18]:
stock_list3 = stock_list1 + stock_list2
#time period
end = dt.datetime.now()
start = end - dt.timedelta(720)
simulations = 10000 
upper_bound = 0.2
steps = 90 
random_seed = 1
portfolio_weights = main(start=start, end=end, stock_list=stock_list3, simulations=simulations, monte_carlo=False, upper_bound=upper_bound, steps=steps, random_seed=random_seed)
portfolio_weights

[*********************100%***********************]  89 of 89 completed
Expected return: 0.081, Risk: 0.127 with Sharpe Ratio:-0.127
optimal weights:
['A', 'ABC', 'ABT', 'ALB', 'AMCR', 'AMGN', 'AMP', 'APD', 'ARE', 'AVB', 'AXP', 'BALL', 'BBY', 'BIIB', 'BLK', 'BMY', 'BSX', 'BXP', 'C', 'CAH', 'CDNS', 'CF', 'CMCSA', 'CNP', 'COF', 'COP', 'CSCO', 'CVS', 'CVX', 'DHR', 'DVN', 'EBAY', 'ECL', 'ED', 'EQIX', 'ES', 'FCX', 'FMC', 'HAS', 'HES', 'HST', 'IRM', 'JNJ', 'JPM', 'JWN', 'KEYS', 'KIM', 'KSS', 'LOW', 'LYB', 'M', 'MAC', 'MOS', 'MPC', 'MRK', 'MRO', 'MS', 'MSFT', 'MTD', 'NI', 'NWL', 'OMC', 'OXY', 'PEG', 'PFE', 'PKG', 'PLD', 'PPG', 'PVH', 'REG', 'RSG', 'SBAC', 'SHW', 'SRE', 'STT', 'SYF', 'TGT', 'TSCO', 'UA', 'UAA', 'V', 'VFC', 'VTR', 'WBA', 'WEC', 'WELL', 'WFC', 'WM', 'XEL']
[0.     0.     0.     0.     0.     0.0828 0.     0.0053 0.     0.
 0.     0.0053 0.     0.     0.     0.1575 0.     0.     0.     0.
 0.     0.0237 0.0069 0.     0.     0.     0.     0.     0.0609 0.
 0.     0.     0.     0.12

,ticker,weight
0,ABC,0.0631
1,ALB,0.0263
2,CDNS,0.0814
3,CF,0.0308
4,DVN,0.0526
5,ED,0.1255
6,IRM,0.0681
7,MPC,0.2000
8,MRK,0.2000
9,OXY,0.0443


In [15]:
uncertain.rename({'ticker2':'ticker'}, axis = 1, inplace = True)
portfolio = pd.concat([long, uncertain], axis = 0)
portfolio = portfolio.merge(portfolio_weights, on='ticker', how='right')
portfolio

,ticker,Name,GICS Sector,ESG,F-Score,weight
0,ABC,AmerisourceBergen,Health Care,48,5.0,0.1200
1,CDNS,Cadence Design Systems,Technology,43,6.0,0.0533
2,CF,CF Industries Holdings,Materials,50,9.0,0.0239
3,DVN,Devon Energy,Energy,54,8.0,0.0484
4,ED,Consolidated Edison,Utilities,42,7.0,0.1364
5,IRM,Iron Mountain,Real Estate,55,NaN,0.0505
6,MPC,Marathon Petroleum,Energy,51,9.0,0.2000
7,MRK,Merck,Health Care,54,7.0,0.2000
8,OXY,Occidental Petroleum,Energy,45,9.0,0.0006
9,RSG,Republic Services,Utilities,40,6.0,0.1569


In [16]:
portfolio.drop(labels = 10, axis = 0, inplace = True)   #duplicate TSCO consumer discretionary

In [17]:
np.sum(portfolio['weight']*portfolio['ESG'])

48.0398